In [5]:
# coding: utf-8
import sys
import ast
from ast import AST
import tokenize
import token
from numbers import Number
import json
import io

In [13]:
code = sys.argv[0]
text = ""
text = ""
tree = ast.parse(text)

SyntaxError: invalid syntax (<ipython-input-13-5a34ff29ca93>, line 2)

In [7]:
#text = text.encode("unicode_escape").decode("utf-8")
test = io.BytesIO(text.encode('utf-8'))

In [8]:
for child in ast.iter_fields(tree):
    print(child)

('body', [])


In [9]:
print(ast.dump(tree, False, True))

Module([])


In [10]:
g = tokenize.tokenize(test.readline)
tk = list(g)
print(tk)

[TokenInfo(type=59 (BACKQUOTE), string='utf-8', start=(0, 0), end=(0, 0), line=''), TokenInfo(type=0 (ENDMARKER), string='', start=(1, 0), end=(1, 0), line='')]


In [10]:
def tokenASTCombine(tok, astNode):
    tok.pop(0)
    nodeyList, looseTokenList, tk = zipTokAST(tok, astNode, []) #skip start marker token
    while(len(tk) > 1): #skip the end maker
        if(tk[0].type != token.NEWLINE):
            nodeyList.append(formatToken(tk[0]))
        tk.pop(0)
    return json.dumps(nodeyList)


def formatToken(tk):
    return {'type': token.tok_name[tk.type], 'start': {'line': tk.start[0], 'col': tk.start[1]}, 'end': {'line': tk.end[0], 'col': tk.end[1]}, 'literal': tk.string}


def zipTokAST(tk, node, nodeyList):
    if(len(tk) < 1 or tk[0].type == token.ENDMARKER):
        return nodeyList, [], [] #end of tokens
    elif(node == None):
        return nodeyList, [], [] #end of nodes
    
    childrenZip = []
    for child in ast.iter_child_nodes(node): #depth fist
        cz, tz, tk = zipTokAST(tk, child, [])
        if(len(tz) > 0):
            childrenZip += tz
        if(len(cz) > 0):
            childrenZip += cz
        
    marker = {'type': type(node).__name__, 'content': childrenZip}
    looseTokenList = []
    
    if(hasattr(node, 'lineno')): #meaning it's a node that appears in the text
        line = node.lineno
        col = node.col_offset
        #print("node is", type(node).__name__, line, col, tk[0])
        # check for nodes that don't belong to anyone
        while(len(tk) > 0 and (tk[0].start[0] < line or (tk[0].start[0] == line and tk[0].start[1] < col))): #actually starts before this node
            if(tk[0].type != token.NEWLINE):
                looseTokenList.append(formatToken(tk[0]))
            tk.pop(0)
        if(len(tk) > 0 and tk[0].start[0] == line and tk[0].start[1] == col):
            if(tk[0].type != token.NEWLINE):
                formatted = formatToken(tk[0])
                marker['start'] = formatted['start']
                marker['end'] = formatted['end']
                marker['literal'] = formatted['literal']
            tk.pop(0)
            
    if('literal' in marker or len(childrenZip) > 0):
        nodeyList.append(marker)
  
    return nodeyList, looseTokenList, tk
        
                

In [38]:
print(tokenASTCombine(tk, tree))


[{"type": "Module", "content": [{"type": "Import", "content": [], "start": {"line": 1, "col": 0}, "end": {"line": 1, "col": 6}, "literal": "import"}, {"type": "NAME", "start": {"line": 1, "col": 7}, "end": {"line": 1, "col": 10}, "literal": "sys"}, {"type": "Import", "content": [], "start": {"line": 2, "col": 0}, "end": {"line": 2, "col": 6}, "literal": "import"}, {"type": "NAME", "start": {"line": 2, "col": 7}, "end": {"line": 2, "col": 10}, "literal": "ast"}, {"type": "ImportFrom", "content": [], "start": {"line": 3, "col": 0}, "end": {"line": 3, "col": 4}, "literal": "from"}, {"type": "NAME", "start": {"line": 3, "col": 5}, "end": {"line": 3, "col": 8}, "literal": "ast"}, {"type": "NAME", "start": {"line": 3, "col": 9}, "end": {"line": 3, "col": 15}, "literal": "import"}, {"type": "NAME", "start": {"line": 3, "col": 16}, "end": {"line": 3, "col": 19}, "literal": "AST"}, {"type": "Import", "content": [], "start": {"line": 4, "col": 0}, "end": {"line": 4, "col": 6}, "literal": "import

In [39]:
def makeNodey(node):
    jsn = {'type': type(node).__name__, 'node_uid': 0, 'content': []}

    if(hasattr(node, 'lineno')): #meaning it's a node that appears in the text
        jsn['line'] = node.lineno
        jsn['col'] = node.col_offset

    for child in ast.iter_child_nodes(node):
        c = makeNodey(child)
        if(c and jsn):
            jsn['content'].append(c)

    for field, value in ast.iter_fields(node):
        #print("fiel?", field, value)
        if(isinstance(value, str) or isinstance(value, Number) or isinstance(value, bool)):
            #print("fiel is", field, value)
            value = json.dumps(value)
            if(len(jsn['content']) > 0): #add as a name Nodey
                lit = {'type': 'Name', 'literal': value, 'node_uid': 0, 'content': [], 'line': node.lineno, 'col': node.col_offset}
                jsn['content'].append(lit)
            else:
                jsn['literal'] = value

    if(len(jsn['content']) > 0 or 'literal' in jsn):
        return jsn

    

In [40]:
jsn = makeNodey(tree)
json.dumps(jsn)

'{"type": "Module", "node_uid": 0, "content": [{"type": "Import", "node_uid": 0, "content": [{"type": "alias", "node_uid": 0, "content": [], "literal": "\\"sys\\""}], "line": 1, "col": 0}, {"type": "Import", "node_uid": 0, "content": [{"type": "alias", "node_uid": 0, "content": [], "literal": "\\"ast\\""}], "line": 2, "col": 0}, {"type": "ImportFrom", "node_uid": 0, "content": [{"type": "alias", "node_uid": 0, "content": [], "literal": "\\"AST\\""}, {"type": "Name", "literal": "\\"ast\\"", "node_uid": 0, "content": [], "line": 3, "col": 0}, {"type": "Name", "literal": "0", "node_uid": 0, "content": [], "line": 3, "col": 0}], "line": 3, "col": 0}, {"type": "Import", "node_uid": 0, "content": [{"type": "alias", "node_uid": 0, "content": [], "literal": "\\"tokenize\\""}], "line": 4, "col": 0}, {"type": "Import", "node_uid": 0, "content": [{"type": "alias", "node_uid": 0, "content": [], "literal": "\\"token\\""}], "line": 5, "col": 0}, {"type": "ImportFrom", "node_uid": 0, "content": [{"ty